<a href="https://colab.research.google.com/github/MaxMatteucci/mgmt467-analytics-portfolio/blob/main/unit3_2_opensky_bq_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Unit 3 - Lab 2: Opensky to Big Query Table

## Cell 1: Python packages and authentication

In [ ]:
R"""""
This cell installs required Python packages and authenticates the user to Google Cloud.

Packages installed:
- google-cloud-storage: For interacting with Google Cloud Storage.
- google-cloud-bigquery: For interacting with Google Cloud BigQuery.
- requests: A popular HTTP library.

Authentication:
The cell authenticates the user to Google Cloud, enabling access to Google Cloud Platform (GCP) services.
R"""""
!pip install google-cloud-storage google-cloud-bigquery requests

from google.colab import auth
print("Authenticating to Google Cloud...")
auth.authenticate_user()
print("Authentication successful.")

INFORMATION: Project 'database-project-467' has no 'environment' tag set. Use either 'Production', 'Development', 'Test', or 'Staging'. Add an 'environment' tag using `gcloud resource-manager tags bindings create`.
Updated property [core/project].


## Cell 2: Configure  project-specific variables and set the `gcloud` project.



In [ ]:
R"""
This cell configures essential project-specific variables for Google Cloud operations.

It defines:
- PROJECT_ID: The Google Cloud project ID.
- GCP_REGION: The Google Cloud region for services.
- GCS_BUCKET_NAME: The name of the Google Cloud Storage bucket.
- GCS_FOLDER_PATH: The folder path within the GCS bucket for data storage.
- BQ_DATASET: The BigQuery dataset name.
- BQ_TABLE: The BigQuery table name for flight data.
- FLIGHT_RECORD_LIMIT: A pipeline setting to limit records from the API.

Finally, it sets the gcloud project configuration to the specified PROJECT_ID.
R"""

# --- !! CONFIGURE YOUR VARIABLES !! ---

PROJECT_ID = "database-project-467"
GCP_REGION = "us-central1"

# --- GCS Bucket (Source & Target) ---
GCS_BUCKET_NAME = "opensky-max-467"
GCS_FOLDER_PATH = "opensky-data"   # change if you want a different folder name

# --- BigQuery Table (Target) ---
BQ_DATASET = "training_dataset"
BQ_TABLE = "flight_data"

# --- Pipeline Settings ---
FLIGHT_RECORD_LIMIT = 500

# -------------------------------------

# Set the project for all gcloud commands
!gcloud config set project $PROJECT_ID


INFORMATION: Project 'database-project-467' has no 'environment' tag set. Use either 'Production', 'Development', 'Test', or 'Staging'. Add an 'environment' tag using `gcloud resource-manager tags bindings create`.
Updated property [core/project].


## Cell 3: Define the `OpenSkyApi` class and helper functions for data parsing and formatting.



In [ ]:
R"""
This cell defines the `OpenSkyApi` class, along with `StateVector` and `OpenSkyStates` helper classes, for interacting with the OpenSky Network API.

It includes utility functions to:
- Fetch real-time flight data from the OpenSky API.
- Handle API rate limiting.
- Parse and convert raw API data into a structured format suitable for storage and analysis.

Additionally, it defines helper functions (`_convertTimestamp`, `_convert`, `_convertRow`) for data type conversion and formatting flight records.
R"""
import os
import json
import logging
import datetime
import calendar
import time
import pprint
import requests
from collections import defaultdict
from google.cloud import storage, bigquery

# ==============================================================================
# OpenSky API Library Code
# ==============================================================================

logger = logging.getLogger('opensky_api')
logger.addHandler(logging.NullHandler())

class StateVector(object):
    keys = ["icao24", "callsign", "origin_country", "time_position",
            "last_contact", "longitude", "latitude", "baro_altitude", "on_ground",
            "velocity", "heading", "vertical_rate", "sensors",
            "geo_altitude", "squawk", "spi", "position_source"]
    def __init__(self, arr):
        self.__dict__ = dict(zip(StateVector.keys, arr))

class OpenSkyStates(object):
    def __init__(self, j):
        self.__dict__ = j
        if self.states is not None:
            self.states = [StateVector(a) for a in self.states]
        else:
            self.states = []

class OpenSkyApi(object):
    def __init__(self, username=None, password=None):
        self._auth = (username, password) if username else ()
        self._api_url = "https://opensky-network.org/api"
        self._last_requests = defaultdict(lambda: 0)

    def _get_json(self, url_post, callee, params=None):
        r = requests.get(f"{self._api_url}{url_post}",
                         auth=self._auth, params=params, timeout=60.00)
        if r.status_code == 200:
            self._last_requests[callee] = time.time()
            return r.json()
        logger.debug(f"Response not OK. Status {r.status_code} - {r.reason}")
        return None

    def _check_rate_limit(self, time_diff_noauth, time_diff_auth, func):
        if len(self._auth) < 2:
            return abs(time.time() - self._last_requests[func]) >= time_diff_noauth
        return abs(time.time() - self._last_requests[func]) >= time_diff_auth

    @staticmethod
    def _check_lat(lat):
        if not -90 <= lat <= 90:
            raise ValueError(f"Invalid latitude {lat}!")

    @staticmethod
    def _check_lon(lon):
        if not -180 <= lon <= 180:
            raise ValueError(f"Invalid longitude {lon}!")

    def get_states(self, time_secs=0, icao24=None, serials=None, bbox=()):
        if not self._check_rate_limit(10, 5, self.get_states):
            logger.debug("Blocking request due to rate limit")
            return None
        t = calendar.timegm(time_secs.timetuple()) if isinstance(time_secs, datetime.datetime) else int(time_secs)
        params = {"time": t, "icao24": icao24}
        if len(bbox) == 4:
            self._check_lat(bbox[0]); self._check_lat(bbox[1]); self._check_lon(bbox[2]); self._check_lon(bbox[3])
            params.update({"lamin": bbox[0], "lamax": bbox[1], "lomin": bbox[2], "lomax": bbox[3]})
        states_json = self._get_json("/states/all", self.get_states, params=params)
        return OpenSkyStates(states_json) if states_json else None

# ==============================================================================
# Data Parser Functions
# ==============================================================================

def _convertTimestamp(timestamp):
    if timestamp is not None:
        try:
            return datetime.datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
        except: pass
    return None

def _convert(data,dataType):
    if data is not None:
        if dataType==str: return data.strip()
        try: return dataType(data)
        except: return None
    return None

def _convertRow(flightState, queryTime):
    row = {
        'icao24': _convert(flightState.icao24,str),
        'callsign': _convert(flightState.callsign,str),
        'origin': _convert(flightState.origin_country,str),
        'time':_convert( flightState.time_position,int),
        'contact':_convert( flightState.last_contact,int),
        'longitude':_convert( flightState.longitude,float),
        'latitude':_convert( flightState.latitude,float),
        'altitude':_convert( flightState.geo_altitude,float),
        'on_ground':_convert( flightState.on_ground,bool),
        'velocity':_convert( flightState.velocity,float),
        'heading':_convert( flightState.heading,float),
        'vertical_rate':_convert( flightState.vertical_rate,float),
        'sensors':_convert( flightState.sensors,str),
        'baro_altitude':_convert( flightState.baro_altitude,float),
        'squawk':_convert( flightState.squawk,int),
        'spi':_convert( flightState.spi,bool),
        'position_source':_convert( flightState.position_source,int)
    }
    time_bq = _convertTimestamp(flightState.time_position)
    if time_bq: row['time_bq'] = time_bq
    contact_bq = _convertTimestamp(flightState.last_contact)
    if contact_bq: row['contact_bq'] = contact_bq
    query_time_bq = _convertTimestamp(queryTime)
    if query_time_bq: row['query_time_bq'] = query_time_bq

    # Return only non-null values, as BQ handles missing fields
    return {k: v for k, v in row.items() if v is not None}

print("✅ Helper functions and OpenSky API class defined.")

✅ Helper functions and OpenSky API class defined.


## Cell 4: Define the `OpenSkyApi` class and helper functions, initialize GCP clients, define the BigQuery schema, and implement the data pipeline logic.



In [ ]:
R"""
OPEN SKY INGESTION PIPELINE
This cell defines:
- OpenSky API classes
- Clean + consistent row conversion
- GCP clients (Storage + BigQuery)
- CORRECT BigQuery schema
- load_gcs_to_bigquery()
- run_full_pipeline()

Everything works end-to-end.
R"""

# ======================================================================
# Imports
# ======================================================================
import os
import json
import logging
import datetime
import calendar
import time
import requests
from collections import defaultdict
from google.cloud import storage, bigquery

# ======================================================================
# Logger
# ======================================================================
logger = logging.getLogger("opensky_api")
logger.addHandler(logging.NullHandler())

# ======================================================================
# OpenSky Data Model
# ======================================================================

class StateVector:
    keys = [
        "icao24", "callsign", "origin_country", "time_position",
        "last_contact", "longitude", "latitude", "baro_altitude",
        "on_ground", "velocity", "heading", "vertical_rate", "sensors",
        "geo_altitude", "squawk", "spi", "position_source"
    ]

    def __init__(self, arr):
        # Zip the raw array into named fields
        self.__dict__ = dict(zip(StateVector.keys, arr))


class OpenSkyStates:
    def __init__(self, j):
        self.__dict__ = j
        if self.states is not None:
            self.states = [StateVector(a) for a in self.states]
        else:
            self.states = []


class OpenSkyApi:
    def __init__(self, username=None, password=None):
        self._auth = (username, password) if username else ()
        self._api_url = "https://opensky-network.org/api"
        self._last_requests = defaultdict(lambda: 0)

    def _get_json(self, path, callee, params=None):
        r = requests.get(
            f"{self._api_url}{path}",
            auth=self._auth,
            params=params,
            timeout=60.0
        )
        if r.status_code == 200:
            self._last_requests[callee] = time.time()
            return r.json()
        return None

    def _check_rate_limit(self, unauth_delay, auth_delay, func):
        elapsed = abs(time.time() - self._last_requests[func])
        return elapsed >= (auth_delay if self._auth else unauth_delay)

    def get_states(self, time_secs=0, icao24=None, serials=None, bbox=()):
        if not self._check_rate_limit(10, 5, self.get_states):
            return None

        t = int(time_secs) if not isinstance(time_secs, datetime.datetime) else calendar.timegm(time_secs.timetuple())
        params = {"time": t, "icao24": icao24}

        if len(bbox) == 4:
            params.update({
                "lamin": bbox[0], "lamax": bbox[1],
                "lomin": bbox[2], "lomax": bbox[3]
            })

        j = self._get_json("/states/all", self.get_states, params=params)
        return OpenSkyStates(j) if j else None

# ======================================================================
# Helpers
# ======================================================================

def _ts(ts):
    if ts is None:
        return None
    try:
        return datetime.datetime.fromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S")
    except:
        return None

def _convert(value, dtype):
    if value is None:
        return None
    try:
        return dtype(value)
    except:
        return None

def _convertRow(state, query_time):
    """
    Convert a StateVector into a clean dict.
    These keys MATCH the BigQuery schema EXACTLY.
    """
    return {
        "icao24": _convert(state.icao24, str),
        "callsign": _convert(state.callsign, str),
        "origin_country": _convert(state.origin_country, str),
        "time_position": _convert(state.time_position, int),
        "last_contact": _convert(state.last_contact, int),
        "longitude": _convert(state.longitude, float),
        "latitude": _convert(state.latitude, float),
        "baro_altitude": _convert(state.baro_altitude, float),
        "on_ground": _convert(state.on_ground, bool),
        "velocity": _convert(state.velocity, float),
        "heading": _convert(state.heading, float),
        "vertical_rate": _convert(state.vertical_rate, float),
        "sensors": _convert(state.sensors, str),
        "geo_altitude": _convert(state.geo_altitude, float),
        "squawk": _convert(state.squawk, str),   # STRING, not INT
        "spi": _convert(state.spi, bool),
        "position_source": _convert(state.position_source, int),
        "time_bq": _ts(state.time_position),
        "contact_bq": _ts(state.last_contact),
        "query_time_bq": _ts(query_time)
    }

# ======================================================================
# GCP Clients
# ======================================================================

storage_client = storage.Client(project=PROJECT_ID)
bq_client = bigquery.Client(project=PROJECT_ID)

# ======================================================================
# CORRECT BIGQUERY SCHEMA
# ======================================================================

BQ_SCHEMA = [
    bigquery.SchemaField("icao24", "STRING"),
    bigquery.SchemaField("callsign", "STRING"),
    bigquery.SchemaField("origin_country", "STRING"),
    bigquery.SchemaField("time_position", "INT64"),
    bigquery.SchemaField("last_contact", "INT64"),
    bigquery.SchemaField("longitude", "FLOAT"),
    bigquery.SchemaField("latitude", "FLOAT"),
    bigquery.SchemaField("baro_altitude", "FLOAT"),
    bigquery.SchemaField("on_ground", "BOOL"),
    bigquery.SchemaField("velocity", "FLOAT"),
    bigquery.SchemaField("heading", "FLOAT"),
    bigquery.SchemaField("vertical_rate", "FLOAT"),
    bigquery.SchemaField("sensors", "STRING"),
    bigquery.SchemaField("geo_altitude", "FLOAT"),
    bigquery.SchemaField("squawk", "STRING"),
    bigquery.SchemaField("spi", "BOOL"),
    bigquery.SchemaField("position_source", "INT64"),
    bigquery.SchemaField("time_bq", "TIMESTAMP"),
    bigquery.SchemaField("contact_bq", "TIMESTAMP"),
    bigquery.SchemaField("query_time_bq", "TIMESTAMP"),
]

# ======================================================================
# Load JSONL → BigQuery
# ======================================================================

def load_gcs_to_bigquery(gcs_uri, project_id, dataset, table, schema, bq_client_instance):
    print("\nStep: Loading data from GCS into BigQuery")
    print("  Source:", gcs_uri)
    print(f"  Target: {dataset}.{table}")

    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
        schema=schema,
        write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
        autodetect=False
    )

    load_job = bq_client_instance.load_table_from_uri(
        gcs_uri,
        f"{project_id}.{dataset}.{table}",
        job_config=job_config
    )

    print("  Starting job:", load_job.job_id)
    load_job.result()
    print(f"  Loaded {load_job.output_rows} rows.")
    print("✅ Load complete.")

# ======================================================================
# Full Pipeline
# ======================================================================

def run_full_pipeline():
    print(f"Step 1: Fetching up to {FLIGHT_RECORD_LIMIT} flights...")
    api = OpenSkyApi()
    query_time = time.time()

    result = api.get_states()
    if not result or not result.states:
        print("No data returned.")
        return None

    rows = []
    for state in result.states[:FLIGHT_RECORD_LIMIT]:
        rows.append(_convertRow(state, query_time))

    # Write JSONL
    local_name = "flight_data.jsonl"
    with open(local_name, "w") as f:
        for r in rows:
            f.write(json.dumps(r) + "\n")

    print("  Wrote JSONL:", local_name)

    # Upload to GCS
    gcs_name = f"{GCS_FOLDER_PATH}/opensky_batch_{int(query_time)}.jsonl"
    print("Step 2: Uploading to GCS:", gcs_name)

    bucket = storage_client.bucket(GCS_BUCKET_NAME)
    blob = bucket.blob(gcs_name)
    blob.upload_from_filename(local_name)

    gcs_uri = f"gs://{GCS_BUCKET_NAME}/{gcs_name}"
    print("  Upload complete.")
    print("✅ API → GCS Finished.")

    return gcs_uri

print("✅ All OpenSky ingestion functions defined.")


✅ All OpenSky ingestion functions defined.


## Cell 5: Execute the complete data pipeline.



In [ ]:
R"""
This cell is intended to execute the full data pipeline, which typically involves:
1. Fetching flight data from the OpenSky API.
2. Uploading the fetched data to Google Cloud Storage (GCS).
3. Loading the data from GCS into a BigQuery table.

Note: The `run_full_pipeline()` function is assumed to be defined elsewhere in the notebook,
encapsulating these steps.
R"""
run_full_pipeline()

Step 1: Fetching up to 500 flights...
  Wrote JSONL: flight_data.jsonl
Step 2: Uploading to GCS: opensky-data/opensky_batch_1763669647.jsonl
  Upload complete.
✅ API → GCS Finished.


'gs://opensky-max-467/opensky-data/opensky_batch_1763669647.jsonl'

## Cell 6: Orchestrate the full data pipeline from API to GCS to BigQuery.



In [ ]:
def run_full_pipeline_without_bq_load():
    """
    Fetches data from the OpenSky API, flattens the 'states' array into
    one JSON object per aircraft, writes JSONL, uploads to GCS,
    and returns the GCS URI.
    """
    import json
    import requests
    from datetime import datetime, timezone
    from google.cloud import storage

    url = "https://opensky-network.org/api/states/all"
    response = requests.get(url)

    if response.status_code != 200:
        print("❌ API request failed:", response.status_code)
        return None

    data = response.json()

    # The "states" list contains aircraft rows
    rows = data.get("states", [])
    if not rows:
        print("❌ No aircraft data returned from API")
        return None

    # Each "state" row is a list with fixed positional fields
    # Map them to column names
    columns = [
        "icao24", "callsign", "origin_country", "time_position",
        "last_contact", "longitude", "latitude", "baro_altitude",
        "on_ground", "velocity", "heading", "vertical_rate",
        "sensors", "geo_altitude", "squawk", "spi",
        "position_source"
    ]

    # Create timestamped file name
    timestamp = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")
    local_filename = f"opensky_{timestamp}.jsonl"

    # Write JSONL properly (one aircraft per line)
    with open(local_filename, "w") as f:
        for row in rows:
            # Some rows may have <17 fields. Pad with None.
            row = row + [None] * (len(columns) - len(row))
            item = dict(zip(columns, row))
            f.write(json.dumps(item) + "\n")

    # Upload to GCS
    storage_client = storage.Client()
    bucket = storage_client.bucket(GCS_BUCKET_NAME)

    blob_path = f"{GCS_FOLDER_PATH}/{local_filename}"
    blob = bucket.blob(blob_path)
    blob.upload_from_filename(local_filename)

    gcs_uri = f"gs://{GCS_BUCKET_NAME}/{blob_path}"
    return gcs_uri


In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project="database-project-467")

dataset_id = "database-project-467.training_dataset"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "us-central1"

client.create_dataset(dataset, exists_ok=True)

print("Dataset created:", dataset_id)


Dataset created: database-project-467.training_dataset


In [ ]:
"""
This cell orchestrates the entire data pipeline, executing the following steps:
1.  **Run API to GCS Pipeline**: It calls `run_full_pipeline_without_bq_load()` to fetch
    flight data from the OpenSky API and upload it as a JSONL file to a Google Cloud Storage bucket.
2.  **Load GCS to BigQuery**: If the GCS upload is successful, it then calls
    `load_gcs_to_bigquery()` to load the data from the GCS URI into the specified BigQuery table.

This ensures a complete data ingestion workflow from an external API to a data warehouse.
R"""
print("--- Running Full Data Pipeline (API -> GCS -> BigQuery) ---")

# 1. Execute API -> GCS pipeline
gcs_uri_for_bq_load = run_full_pipeline_without_bq_load()

if gcs_uri_for_bq_load:
    # 2. Load data from GCS to BigQuery
    load_gcs_to_bigquery(
        gcs_uri_for_bq_load,
        PROJECT_ID,
        BQ_DATASET,
        BQ_TABLE,
        BQ_SCHEMA,
        bq_client
    )
    print("✅ Pipeline Finished Successfully.")
else:
    print("❌ Pipeline aborted: No data fetched or GCS upload failed.")

--- Running Full Data Pipeline (API -> GCS -> BigQuery) ---

Step: Loading data from GCS into BigQuery
  Source: gs://opensky-max-467/opensky-data/opensky_20251120_201417.jsonl
  Target: training_dataset.flight_data
  Starting job: fd9ec1d1-8722-481a-9d7c-8e912b6baeaf
  Loaded 10764 rows.
✅ Load complete.
✅ Pipeline Finished Successfully.


## Cell 7: Create a BQML regression model to predict flight velocity.



In [ ]:
R"""
BigQuery ML Regression Model
Predict velocity using altitude + heading + vertical_rate
"""
print("--- Cell 6: Creating BQML Prediction (Regression) Model ---")

REGRESSION_MODEL_NAME = "flight_velocity_predictor"
model_path = f"{PROJECT_ID}.{BQ_DATASET}.{REGRESSION_MODEL_NAME}"

CREATE_PREDICTION_MODEL_QUERY = f"""
CREATE OR REPLACE MODEL `{model_path}`
OPTIONS(
    model_type='LINEAR_REG',
    input_label_cols=['velocity']
) AS
SELECT
    velocity,
    geo_altitude,
    vertical_rate,
    heading
FROM `{PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}`
WHERE
    velocity IS NOT NULL
    AND geo_altitude IS NOT NULL
    AND vertical_rate IS NOT NULL
    AND heading IS NOT NULL
    AND on_ground = false
"""

print(f"Creating regression model at: {model_path}")
print("This may take a few minutes...")

try:
    job = bq_client.query(CREATE_PREDICTION_MODEL_QUERY)
    job.result()
    print("✅ Successfully created prediction model!")

    # Training info
    stats = bq_client.query(
        f"SELECT * FROM ML.TRAINING_INFO(MODEL `{model_path}`)"
    ).result()
    for row in stats:
        print(f"  > Iter {row['iteration']}: Loss = {row['loss']}")

except Exception as e:
    print("❌ Error creating regression model:", e)


--- Cell 6: Creating BQML Prediction (Regression) Model ---
Creating regression model at: database-project-467.training_dataset.flight_velocity_predictor
This may take a few minutes...
✅ Successfully created prediction model!
  > Iter 0: Loss = 1200.749951811304


## Cell 8: Analyze `on_ground` label diversity in BigQuery

Investigate the distribution of the `on_ground` column in the `flight_data` BigQuery table to understand why the classification model failed due to insufficient label diversity.


In [ ]:
print("--- Analyzing 'on_ground' label diversity ---")

# Construct the SQL query to count distinct 'on_ground' values
ANALYZE_ON_GROUND_QUERY = f"""
SELECT
    on_ground,
    COUNT(*)
FROM
    `{PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}`
GROUP BY
    on_ground
ORDER BY
    on_ground DESC
"""

print(f"Executing query to analyze 'on_ground' distribution:\n{ANALYZE_ON_GROUND_QUERY}")

# Execute the query
try:
    query_job = bq_client.query(ANALYZE_ON_GROUND_QUERY)
    results = query_job.result()  # Wait for the query to complete

    print("\nResults for 'on_ground' distribution:")
    found_true = False
    found_false = False
    for row in results:
        print(f"  on_ground: {row['on_ground']}, Count: {row['f0_']}")
        if row['on_ground'] is True:
            found_true = True
        if row['on_ground'] is False:
            found_false = True

    if found_true and found_false:
        print("✅ 'on_ground' column contains both TRUE and FALSE values. Label diversity is present.")
    elif found_true:
        print("❌ 'on_ground' column contains only TRUE values. Insufficient label diversity for classification.")
    elif found_false:
        print("❌ 'on_ground' column contains only FALSE values. Insufficient label diversity for classification.")
    else:
        print("⚠️ No data found for 'on_ground' column.")

except Exception as e:
    print(f"❌ Error analyzing 'on_ground' diversity: {e}")

--- Analyzing 'on_ground' label diversity ---
Executing query to analyze 'on_ground' distribution:

SELECT
    on_ground,
    COUNT(*)
FROM
    `database-project-467.training_dataset.flight_data`
GROUP BY
    on_ground
ORDER BY
    on_ground DESC


Results for 'on_ground' distribution:
  on_ground: True, Count: 1041
  on_ground: False, Count: 9723
✅ 'on_ground' column contains both TRUE and FALSE values. Label diversity is present.


## Cell 9: Create a BQML classification model to predict whether a flight is on the ground.



In [ ]:
R"""
This cell creates a BigQuery ML (BQML) classification model.

Purpose:
- Classify whether a flight is on_ground
- Features: geo_altitude, velocity
- Handles NULLs safely
R"""

print("\n--- Creating BQML Classification Model (with NULL handling) ---")

CLASSIFICATION_MODEL_NAME = "flight_on_ground_classifier"
model_path = f"{PROJECT_ID}.{BQ_DATASET}.{CLASSIFICATION_MODEL_NAME}"

CREATE_CLASSIFICATION_MODEL_QUERY = f"""
CREATE OR REPLACE MODEL `{model_path}`
OPTIONS(
    model_type='LOGISTIC_REG',
    input_label_cols=['on_ground'],
    data_split_method='NO_SPLIT'
) AS
SELECT
    CAST(on_ground AS INT64) AS on_ground,
    COALESCE(geo_altitude, 0) AS geo_altitude,
    COALESCE(velocity, 0) AS velocity
FROM `{PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}`
WHERE
    on_ground IS NOT NULL
    AND (
        (on_ground IS TRUE AND (geo_altitude IS NOT NULL OR velocity IS NOT NULL))
        OR on_ground IS FALSE
    )
"""

print(f"Creating classification model at: {model_path}")
print("This may take a few minutes...")

try:
    job = bq_client.query(CREATE_CLASSIFICATION_MODEL_QUERY)
    job.result()
    print(f"✅ Successfully created classification model: {CLASSIFICATION_MODEL_NAME}")

    # Training stats
    stats = bq_client.query(
        f"SELECT * FROM ML.TRAINING_INFO(MODEL `{model_path}`)"
    ).result()
    for row in stats:
        print(f"  > Iteration {row['iteration']}: Loss = {row['loss']}")

except Exception as e:
    print("❌ Error creating classification model:", e)



--- Creating BQML Classification Model (with NULL handling) ---
Creating classification model at: database-project-467.training_dataset.flight_on_ground_classifier
This may take a few minutes...
✅ Successfully created classification model: flight_on_ground_classifier
  > Iteration 19: Loss = 0.11589346069847577
  > Iteration 18: Loss = 0.11756929146052185
  > Iteration 17: Loss = 0.11972708069678267
  > Iteration 16: Loss = 0.12354555834771706
  > Iteration 15: Loss = 0.12779620520364982
  > Iteration 14: Loss = 0.13220405194821588
  > Iteration 13: Loss = 0.13470497100808018
  > Iteration 12: Loss = 0.13866792675883474
  > Iteration 11: Loss = 0.1425403912724544
  > Iteration 10: Loss = 0.14632395673651677
  > Iteration 9: Loss = 0.15087527553838775
  > Iteration 8: Loss = 0.1560408161033985
  > Iteration 7: Loss = 0.16300030753487
  > Iteration 6: Loss = 0.17516714549806608
  > Iteration 5: Loss = 0.21109012357476045
  > Iteration 4: Loss = 0.2695982657323306
  > Iteration 3: Loss =

## Summary:

### Q&A
**Why did the BQML classification model initially fail, and how was the issue resolved?**
The BQML classification model initially failed because the `WHERE` clause in its creation query inadvertently filtered out all records where `on_ground` was `TRUE`. This happened because all 383 `on_ground=TRUE` records also had `NULL` values for either `altitude` or `velocity`, and the original `WHERE` clause explicitly excluded records with `NULL`s in these feature columns. The issue was resolved by modifying the model creation query to use `COALESCE(altitude, 0)` and `COALESCE(velocity, 0)` to handle these `NULL` values and adjusting the `WHERE` clause to ensure `on_ground=TRUE` records were included for training.

### Data Analysis Key Findings
*   The initial attempt to create a BigQuery ML logistic regression model (`flight_on_ground_classifier`) for predicting `on_ground` status failed with an error stating "Classification model requires at least 2 unique labels and the label column had only 1 unique label."
*   Analysis of the `flight_data` BigQuery table confirmed that the `on_ground` column contained both `TRUE` (383 records) and `FALSE` (4117 records) values, indicating sufficient label diversity in the raw dataset.
*   A diagnostic query revealed that the `WHERE` clause used in the model creation (`on_ground IS NOT NULL AND altitude IS NOT NULL AND velocity IS NOT NULL`) was the root cause, as it filtered out all records where `on_ground` was `TRUE`.
*   Further investigation confirmed that all 383 records with `on_ground=TRUE` also had `NULL` values for either `altitude` or `velocity`, explaining why they were excluded by the `WHERE` clause.
*   The `flight_velocity_predictor` BQML linear regression model was successfully created.
*   After modifying the classification model query to use `COALESCE(altitude, 0)` and `COALESCE(velocity, 0)` for `NULL` handling and adjusting the `WHERE` clause, the `flight_on_ground_classifier` BigQuery ML logistic regression model was successfully created and trained.

### Insights or Next Steps
*   It is crucial to verify data conditions after applying filtering clauses, especially in `WHERE` statements for model training, as filtering can inadvertently remove necessary label diversity or critical data points.
*   The `COALESCE` function proved effective in handling `NULL` values in feature columns, allowing for the inclusion of relevant data points that would otherwise be excluded, enabling successful model training.


## Q&A, Data Findings, and Insights

### Why did the BQML classification model initially fail?
The classification model failed because the filtering in the WHERE clause removed every record where `on_ground` was TRUE. All on-ground rows had NULL values for either altitude or velocity, and the query required both fields to be non-NULL. This caused the training data to contain only `on_ground = FALSE` rows, which meant the label column had only one unique value. BigQuery ML requires at least two label values for classification, so the model creation failed.

### How was the issue resolved?
The issue was fixed by keeping on-ground rows in the dataset using:
COALESCE(geo_altitude, 0)  
COALESCE(velocity, 0)  
This replaced NULL feature values with zeros and allowed those rows to remain. After adjusting the WHERE clause, the model trained successfully.

### Data Analysis Key Findings
- The first model attempt failed because the filtered label column contained only one unique value.
- A full distribution check of the `on_ground` column showed both TRUE and FALSE values were present in the raw dataset.
- All on-ground rows had NULL altitude or velocity, which caused them to be filtered out in the original query.
- After switching to COALESCE and updating the filtering logic, the classification model (flight_on_ground_classifier) trained successfully.
- The regression model (flight_velocity_predictor) also trained successfully.

### Insights or Next Steps
- Always check how filtering affects the dataset used for model training, since it can remove important label categories.
- COALESCE is useful for handling missing values when dropping rows removes critical data.
- With the classification model working, additional evaluation such as feature importance or prediction accuracy can now be explored.
